# Notebook that handles fine-tuning from start to finish

In [2]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-0mxpozam
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-0mxpozam
  Resolved https://github.com/huggingface/transformers.git to commit 4a27c13f1eee26393d60d381e500e1a61970e8ee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
  Using cached regex-2023.8.8-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (771 kB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached safetensors-0.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7476448 sha256=265cd3e22309e718

In [3]:
!pip install accelerate==0.20.3

  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.16.0
    Uninstalling accelerate-0.16.0:
      Successfully uninstalled accelerate-0.16.0

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install datasets

  Using cached datasets-2.14.4-py3-none-any.whl (519 kB)
  Using cached aiohttp-3.8.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Using cached xxhash-3.3.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (193 kB)
  Using cached yarl-1.9.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (269 kB)
  Using cached multidict-6.0.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (114 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached frozenlist-1.4.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (228 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import numpy as np
from sagemaker import get_execution_role
import boto3
import pandas as pd
from io import StringIO # Python 3.
from datasets import load_dataset,Dataset,DatasetDict,concatenate_datasets

from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import json

from models.EDdisposition import EDdispositionClassifier

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# reads out files

import boto3

s3 = boto3.resource('s3')
bucket = s3.Bucket('chianglab-dataderivatives')

folders = set()

for obj in bucket.objects.all():
    prefix, delimiter, _ = obj.key.rpartition('/')
    if prefix:
        folders.add(prefix + '/')

print('Folders:')
for folder in folders:
    print(folder)
    


Folders:
mimic-iv-clinical-database-demo-2.2/
mimic-iv-2.2/
mimic-iv-ed-2.2/
mimic-iv-ed-demo-2.2/


In [7]:
bucket = 'chianglab-dataderivatives'
subfolder = 'mimic-iv-ed-demo-2.2/'
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

file_list = []

for f in contents:
    print(f['Key'])
    file_list.append(f['Key'][36:])

print(file_list)

mimic-iv-ed-demo-2.2/
mimic-iv-ed-demo-2.2/text_repr.json
['', '']


### Load in Data and reformat 

In [8]:
s3 = boto3.resource('s3')
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/admissions_text.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)

df = pd.DataFrame(list(json_content.items()), columns = ['ID', 'ad_headline'])

In [9]:
# add codes 

bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/codes_text.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df['codes_headline'] = df['ID'].map(json_content)

# add meds
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/meds_text.json"

content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df['meds_headline'] = df['ID'].map(json_content)

# add labs
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/labs_text.json"

content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
# need to change float IDs to int but then back to a string to map
res = dict()
for key in json_content:
    res[str(int(float(key)))] = json_content[key]
    
df['labs_headline'] = df['ID'].map(res)

### Data we will be working with

In [10]:
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-ed-demo-2.2/text_repr.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df = pd.DataFrame(json_content).T
print("length of dataframe: "+ str(len(df)))
df.head(5)
# df['codes_headline'] = df['ID'].map(json_content)


length of dataframe: 210


,arrival,eddischarge,admission,discharge,triage,medrecon,vitals,pyxis,codes
37887480,"Patient 10014729, a 21 year old white - other ...",The ED disposition was admitted at 2125-03-19 ...,The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...
34176810,"Patient 10018328, a 83 year old white female, ...",The ED disposition was admitted at 2154-02-05 ...,The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,NaN,The patient received the following diagnostic ...
32103106,"Patient 10018328, a 83 year old white female, ...",The ED disposition was home at 2154-08-03 22:2...,The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...
38797992,"Patient 10020640, a 91 year old white female, ...",The ED disposition was admitted at 2153-02-13 ...,The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...
33473053,"Patient 10015272, a 78 year old white female, ...",The ED disposition was admitted at 2137-06-12 ...,The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...


In [11]:
df['eddischarge'] = [1 if 'admitted' in s.lower() else 0 for s in df['eddischarge']] # admitted = 1, Home = 0
df = df[[col for col in df.columns if col != 'eddischarge'] + ['eddischarge']] # rearrange column to the end
df = df.fillna("[PAD]")
df

,arrival,admission,discharge,triage,medrecon,vitals,pyxis,codes,eddischarge
37887480,"Patient 10014729, a 21 year old white - other ...",The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34176810,"Patient 10018328, a 83 year old white female, ...",The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,[PAD],The patient received the following diagnostic ...,1
32103106,"Patient 10018328, a 83 year old white female, ...",The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...,0
38797992,"Patient 10020640, a 91 year old white female, ...",The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...,1
33473053,"Patient 10015272, a 78 year old white female, ...",The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...,1
...,...,...,...,...,...,...,...,...,...
30272878,"Patient 10038999, a 45 year old white male, ar...",The patient was admitted at 2131-05-22 21:49:00.,The patient's discharge disposition was: home ...,"At triage: temperature was not recorded, pulse...",[PAD],The patient had the following vitals: At 2131-...,The patient received the following medications...,The patient received the following diagnostic ...,1
31628990,"Patient 10009049, a 56 year old white male, ar...",The patient was admitted at 2174-05-26 08:21:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.0, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2174-...,The patient received the following medications...,The patient received the following diagnostic ...,1
32405286,"Patient 10004457, a 65 year old white male, ar...",The patient was admitted at 2141-08-12 16:02:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.6, pulse was 103...",The patient was previously taking the followin...,The patient had the following vitals: At 2141-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34391979,"Patient 10004720, a 61 year old white male, ar...",The patient was admitted at 2186-11-12 18:01:00.,The patient's discharge disposition was: died ...,"At triage: temperature was not recorded, pulse...",The patient was previously taking the followin...,The patient had the following vitals: At 2186-...,The patient received the following medications...,The patient received the following diagnostic ...,1


In [12]:
col_names = df.columns.drop("eddischarge")
l = []
for i in col_names:
    temp = df[[i, 'eddischarge']].reset_index()
    temp = temp.sort_values(by=['index']).reset_index() # we sort the patient ID numerically before dropping it to preserve order in encoding
    temp = temp.drop(columns=["index", "level_0"])
    temp = temp.rename(columns={i: "headline", "eddischarge": "label"})
    l.append(temp)
    print("\""+i+ "\" Dataframe")
    display(temp)

"arrival" Dataframe


,headline,label
0,"Patient 10014354, a 60 year old white male, ar...",1
1,"Patient 10010867, a 28 year old white - brazil...",1
2,"Patient 10002930, a 48 year old black/african ...",1
3,"Patient 10005866, a 57 year old portuguese mal...",1
4,"Patient 10038999, a 45 year old white male, ar...",1
...,...,...
205,"Patient 10023117, a 53 year old white male, ar...",1
206,"Patient 10002930, a 48 year old black/african ...",0
207,"Patient 10020740, a 56 year old white male, ar...",0
208,"Patient 10015860, a 53 year old white male, ar...",0


"admission" Dataframe


,headline,label
0,The patient was admitted at 2147-11-14 22:12:00.,1
1,The patient was admitted at 2147-12-30 08:40:00.,1
2,The patient was admitted at 2198-04-17 19:38:00.,1
3,The patient was admitted at 2149-09-20 14:30:00.,1
4,The patient was admitted at 2131-05-22 21:49:00.,1
...,...,...
205,The patient was admitted at 2174-06-07 23:25:00.,1
206,The patient was admitted at 2201-02-12 16:58:00.,0
207,The patient was admitted at 2150-09-15 14:09:00.,0
208,The patient was admitted at 2191-01-15 01:55:00.,0


"discharge" Dataframe


...

,headline,label
0,The patient had the following vitals: At 2147-...,1
1,The patient had the following vitals: At 2147-...,1
2,The patient had the following vitals: At 2198-...,1
3,The patient had the following vitals: At 2149-...,1
4,The patient had the following vitals: At 2131-...,1
...,...,...
205,The patient had the following vitals: At 2174-...,1
206,The patient had the following vitals: At 2201-...,0
207,The patient had the following vitals: At 2150-...,0
208,The patient had the following vitals: At 2191-...,0


"pyxis" Dataframe


,headline,label
0,The patient received the following medications...,1
1,The patient received the following medications...,1
2,The patient received the following medications...,1
3,The patient received the following medications...,1
4,The patient received the following medications...,1
...,...,...
205,[PAD],1
206,The patient received the following medications...,0
207,The patient received the following medications...,0
208,The patient received the following medications...,0


"codes" Dataframe


,headline,label
0,The patient received the following diagnostic ...,1
1,The patient received the following diagnostic ...,1
2,The patient received the following diagnostic ...,1
3,The patient received the following diagnostic ...,1
4,The patient received the following diagnostic ...,1
...,...,...
205,The patient received the following diagnostic ...,1
206,The patient received the following diagnostic ...,0
207,The patient received the following diagnostic ...,0
208,The patient received the following diagnostic ...,0


In [13]:
# Indices for sub-dataframes, that will each individually be tokenized
# 0 - arrival
# 1 - admission
# 2 - discharge
# 3 - triage
# 4 - medrecon
# 5 - vitals
# 6 - pyxis
# 7 - codes
arrival_hf=Dataset.from_pandas(l[0])
admission_hf=Dataset.from_pandas(l[1])
discharge_hf=Dataset.from_pandas(l[2])
triage_hf=Dataset.from_pandas(l[3])
medrecon_hf=Dataset.from_pandas(l[4])
vitals_hf=Dataset.from_pandas(l[5])
pyxis_hf=Dataset.from_pandas(l[6])
codes_hf=Dataset.from_pandas(l[7])

# Sanity check printer
printer = [arrival_hf, admission_hf, discharge_hf, triage_hf, medrecon_hf, vitals_hf, pyxis_hf, codes_hf]
for i in printer:
    print(i)

Dataset({
    features: ['headline', 'label'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label'],
    num_rows: 210
})


In [14]:
# grabs a tokenizer from the distilbert-base-uncased model
model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer.model_max_len=512

In [15]:
def tokenize(batch):
  return tokenizer(batch["headline"], truncation=True, max_length=512)


tokenized_arrival = arrival_hf.map(tokenize, batched=True)
tokenized_admission = admission_hf.map(tokenize, batched=True)
tokenized_discharge = discharge_hf.map(tokenize, batched=True)
tokenized_triage = triage_hf.map(tokenize, batched=True)
tokenized_medrecon = medrecon_hf.map(tokenize, batched=True)
tokenized_vitals = vitals_hf.map(tokenize, batched=True)
tokenized_pyxis = pyxis_hf.map(tokenize, batched=True)
tokenized_codes = codes_hf.map(tokenize, batched=True)

# Sanity check printer
printer = [tokenized_arrival, tokenized_admission, tokenized_discharge, tokenized_triage, tokenized_medrecon, tokenized_vitals, tokenized_pyxis, tokenized_codes]
for i in printer:
    print(i)
    
dataset_cc = concatenate_datasets([tokenized_arrival, tokenized_admission, tokenized_discharge, tokenized_triage, tokenized_medrecon, tokenized_vitals, tokenized_pyxis, tokenized_codes])
print(dataset_cc)
    


Map: 100%|██████████| 210/210 [00:00<00:00, 11962.57 examples/s]

Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1680
})


In [16]:
tokenized_arrival['input_ids'][0]

[101,
 5776,
 2531,
 16932,
 19481,
 2549,
 1010,
 1037,
 3438,
 2095,
 2214,
 2317,
 3287,
 1010,
 3369,
 3081,
 10771,
 2012,
 19936,
 2581,
 1011,
 2340,
 1011,
 2403,
 2324,
 1024,
 2570,
 1024,
 4002,
 1012,
 1996,
 5776,
 1005,
 1055,
 23143,
 3570,
 2003,
 2309,
 1012,
 1996,
 5776,
 1005,
 1055,
 5427,
 2003,
 2060,
 1012,
 1996,
 5776,
 1005,
 1055,
 2653,
 2003,
 2394,
 1012,
 102]

In [17]:
tokenized_admission['input_ids'][0]

[101,
 1996,
 5776,
 2001,
 4914,
 2012,
 19936,
 2581,
 1011,
 2340,
 1011,
 2403,
 2570,
 1024,
 2260,
 1024,
 4002,
 1012,
 102]

In [18]:
dataset_cc.set_format('torch', columns=["input_ids", "attention_mask", "label"] )
print(dataset_cc)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Dataset({
    features: ['headline', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1680
})


In [19]:
train_testvalid = dataset_cc.train_test_split(test_size=0.2,seed=15)


test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=15)

dataset_cc = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})


In [20]:
dataset_cc

DatasetDict({
    train: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1344
    })
    test: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 168
    })
    valid: Dataset({
        features: ['headline', 'label', 'input_ids', 'attention_mask'],
        num_rows: 168
    })
})

In [21]:
class EDdispositionClassifier(nn.Module):
    """
    A task-specific custom transformer model for predicting ED Disposition. 
    This model loads a pre-trained transformer model and adds a new dropout 
    and linear layer at the end for fine-tuning and prediction on specific tasks.
    """
    def __init__(self, checkpoint, num_labels ):
        """
        Args:
            checkpoint (str): The name of the pre-trained model or path to the model weights.
            num_labels (int): The number of output labels in the final classification layer.
        """
        super(EDdispositionClassifier, self).__init__()
        self.num_labels = num_labels # number of labels for classifier
        
        # checkpoint is the model name 
        self.model = model = AutoModel.from_pretrained(checkpoint, config = AutoConfig.from_pretrained(checkpoint, 
                                                                                                       output_attention = True, 
                                                                                                       output_hidden_state = True ) )
        # New Layer
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, num_labels) #FC Layer
        
    def forward(self, input_ids = None, attention_mask=None, labels = None ):
        """
        Forward pass for the model.
        
        Args:
            input_ids (torch.Tensor, optional): Tensor of input IDs. Defaults to None.
            attention_mask (torch.Tensor, optional): Tensor for attention masks. Defaults to None.
            labels (torch.Tensor, optional): Tensor for labels. Defaults to None.
            
        Returns:
            TokenClassifierOutput: A named tuple with the following fields:
            - loss (torch.FloatTensor of shape (1,), optional, returned when label_ids is provided) – Classification loss.
            - logits (torch.FloatTensor of shape (batch_size, num_labels)) – Classification scores before SoftMax.
            - hidden_states (tuple(torch.FloatTensor), optional, returned when output_hidden_states=True is passed or when config.output_hidden_states=True) – Tuple of torch.FloatTensor (one for the output of the embeddings + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).
            - attentions (tuple(torch.FloatTensor), optional, returned when output_attentions=True is passed or when config.output_attentions=True) – Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).
        """
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask  )
        
        last_hidden_state = outputs[0]
        
        sequence_outputs = self.dropouts(last_hidden_state)
        
        logits = self.classifier(sequence_outputs[:, 0, : ].view(-1, 768 ))
        
        loss = None
        if labels is not None:
            loss_func = nn.CrossEntropyLoss() # Change this if it becomes more than binary classification
            loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))
            
            # TokenClassifierOutput
            return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)
        

In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset_cc['train'], shuffle = True, batch_size = 32, collate_fn = data_collator
)

eval_dataloader = DataLoader(
    dataset_cc['valid'], shuffle = True, collate_fn = data_collator
)


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_task_specific = EDdispositionClassifier(checkpoint=model, num_labels=2 ).to(device)

In [25]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model_task_specific.parameters(), lr = 5e-5 )

num_epoch = 3

num_training_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer = optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps,
    
)

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:
from datasets import load_metric
metric = load_metric("f1")

<ipython-input-24-e4b2221a8929>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


In [ ]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epoch * len(eval_dataloader) ))


for epoch in range(num_epoch):
    model_task_specific.train()
    for batch in train_dataloader:
        batch = { k: v.to(device) for k, v in batch.items() }
        outputs = model_task_specific(**batch)
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)
        
    model_task_specific.eval()
    for batch in eval_dataloader:
        batch = { k: v.to(device) for k, v in batch.items() }
        with torch.no_grad():
            outputs = model_task_specific(**batch)
            
        logits = outputs.logits
        predictions = torch.argmax(logits, dim = -1 )
        metric.add_batch(predictions = predictions, references = batch['labels'] )
        progress_bar_eval.update(1)
        
    print(metric.compute()) 

  0%|          | 0/504 [00:00<?, ?it/s]

[2023-08-17 23:17:56.549 pytorch-1-13-cpu-py39-ml-c5-xlarge-982af48816d5ea91e6bea6f27295:52 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-08-17 23:17:56.596 pytorch-1-13-cpu-py39-ml-c5-xlarge-982af48816d5ea91e6bea6f27295:52 INFO profiler_config_parser.py:111] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
